In [1]:
from bs4 import BeautifulSoup
import requests
import re
import requests
import random
from tqdm import tqdm
import requests
from stem import Signal
from stem.control import Controller
import time
import unicodedata 
import json 
import string
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

url='https://www.bspp.org.uk/education/the-big-quiz-all-plant-doctor-questions/'

In [2]:
# List of User-Agent headers
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:119.0) Gecko/20100101 Firefox/119.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Version/13.1.2 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 12; Pixel 6 Build/SP1A.210812.016) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Version/16.1 Mobile/15E148 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36 OPR/70.0.3728.95',
    'Mozilla/5.0 (Linux; Android 11; SM-G991B) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/14.0 Chrome/91.0.4472.124 Mobile Safari/537.36',
    'Mozilla/5.0 (Android 11; Mobile; rv:94.0) Gecko/94.0 Firefox/94.0'
]

# Set up Tor proxy
proxies = {
    'http': 'socks5h://127.0.0.1:9050',
    'https': 'socks5h://127.0.0.1:9050'
}

vqas = dict()
headers = {
        'User-Agent': random.choice(user_agents),
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }

def change_tor_ip():
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()  # Authenticate to the Tor network
        controller.signal(Signal.NEWNYM)  # Request a new IP
    time.sleep(1)  # Wait for the new IP to be assigned    
    
def scraper():
    # Randomly select a User-Agent
    change_tor_ip()
    headers = {
        'User-Agent': random.choice(user_agents),
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }
    response = requests.get(url, headers=headers)
    # print(f"Current IP: {response.json()['origin']}") # Get current IP
    content = ''
    if response.status_code == 200:
        content = response.content
    # Parse the HTML content
    soup = BeautifulSoup(content, "html.parser")
    # Find the quiz container
    quiz_div = soup.find("div", class_="hdq_quiz")

    if quiz_div:
        # Extract question ID
        question_div = quiz_div.find_all("div", class_="hdq_question")
        for question in tqdm(question_div):
            question_id = question.get("id", "").replace("hdq_question_", "")
            
            # Extract question text
            question_text = question.find("h3", class_="hdq_question_heading").get_text(separator=' ',strip=True)
            # Regex pattern to match "#number. other_text"
            pattern = r"#(\d+)\.\s*(.*)"

            # Search for the pattern
            match = re.search(pattern, question_text)

            if match:
                # question_id = match.group(1)  # Capture the number (if needed)
                question_text = match.group(2)  # Capture the rest of the text
            else:
                print("No match found.")
            # Extract image URLs
            images = []
            image_div = question.find("div", class_="hdq_question_featured_image")
            if image_div:
                imgs_tag = image_div.find_all("img", class_="hdq_featured_image")
                if imgs_tag:
                    for img_tag in imgs_tag:
                        # Get the 'data-srcset' attribute which contains the different image sizes
                        srcset = img_tag.get('data-srcset', '')

                        # Split the srcset by commas (each part contains an image URL with its width)
                        srcset_urls = srcset.split(', ')

                        # Sort the image URLs based on the width (in descending order) and pick the first one (best quality)
                        sorted_urls = sorted(srcset_urls, key=lambda x: int(x.split(' ')[1].replace('w', '')) if len(x.split(' '))==2 else '0', reverse=True)

                        # The best image URL is the first one after sorting
                        best_image_url = sorted_urls[0].split(' ')[0]
                        images.append(best_image_url)
            
            # Extract options
            options = []
            correct_answer = ""
            answers_div = question.find("div", class_="hdq_answers")
            for label in answers_div.find_all("label", class_="hdq_label_answer"):
                option_text = label.find("span", class_="hdq_aria_label").get_text(separator=' ', strip=True)
                options.append(option_text)
                if label.find("input", {'class': 'hdq_option hdq_check_input'}).get('value') == "1":
                    correct_answer = option_text
            
            # Extract correct answer
            explanation_section = question.find("div", class_="hdq_question_after_text")
            explanation = ""
            # Extract all <p> tags in the explanation, excluding those with copyright information
            explanation_text = []
            if explanation_section:
                # first_row_text = explanation_section.find("p").get_text(separator=' ',strip=True)
                # if "Confused" in first_row_text:
                #     correct_answer = "Confused"
                # elif "Abiotic" in first_row_text:
                #     correct_answer = "Abiotic"
                # elif "Biotic" in first_row_text:
                #     correct_answer = "Biotic"
                # elif "Not sure" in first_row_text:
                #     correct_answer = "Not sure"
                # else:
                #     correct_answer = explanation_section.find("p").find_next("p").get_text(separator=' ',strip=True).replace("The correct answer is:", "").strip()
                for p in explanation_section.find_all("p"):
                    p_text = p.get_text(separator=' ',strip=True)
                    # Skip paragraphs that contain copyright or similar keywords
                    if any(keyword in p_text.lower() for keyword in ['copyright', '\u00a9', '©', 'credit']):
                        continue
                    explanation_text.append(p_text)
                if explanation_text:
                    explanation = " ".join(explanation_text)

            question_text = unicodedata.normalize('NFKD', question_text).encode('ascii', 'ignore').decode('ascii')
            explanation = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('ascii')
            options = [unicodedata.normalize('NFKD', option).encode('ascii', 'ignore').decode('ascii') for option in options]
            correct_answer = unicodedata.normalize('NFKD', correct_answer).encode('ascii', 'ignore').decode('ascii')
            options = [option.translate(str.maketrans('', '', string.punctuation)) for option in options]
            correct_answer = correct_answer.translate(str.maketrans('', '', string.punctuation))

            for option in options:
                if option in correct_answer:
                    correct_answer = option
                    break
            
            # Create the output dictionary
            output = {
                "id": f"{question_id}",
                "question": question_text,
                "options": options,
                "correct_answer": correct_answer,
                "images": images,
                "explanation": explanation,
                "metadata": {
                    "source": "https://www.bspp.org.uk/education/the-big-quiz-all-plant-doctor-questions/",  # Replace with actual source
                    "license": "All rights reserved. Content © The British Society for Plant Pathology (BSPP)",  # Replace with actual license
                    "author": "BSPP - The British Society for Plant Pathology",  # Replace with actual author
                    "title": "The Big Plant Pathology Quiz"
                }
            }
            vqas[question_id] = output
    else:
        print("Quiz page not found!")

In [3]:
num_iter = 1

for i in range(num_iter):
    scraper()
    print(f"Iteration {i+1} completed. Length of vqas: {len(vqas)}")

100%|██████████| 370/370 [00:00<00:00, 1571.13it/s]

Iteration 1 completed. Length of vqas: 370


In [6]:
# save to json
with open('/workdir/the_big_plant_pathology_quiz.json', 'w') as json_file:
    json.dump(vqas, json_file, indent=4)

In [5]:
vqas

{'364431': {'id': '364431',
  'question': 'This organism has quite the history with uses described in Greek mythology, pagan rituals as well as being used in Christmas decorations. What is it?',
  'options': ['Witches broom',
   'Mistletoe',
   'Epicormic growth',
   'I dont know and dont want to guess'],
  'correct_answer': 'Mistletoe',
  'images': ['https://www.bspp.org.uk/wp-content/uploads/2023/12/a-tree-with-green-leaves-description-automaticall.jpeg'],
  'explanation': 'Answer: Mistletoe (Viscum album) is a plant parasite with a broad host range . Most commonly found on apple it can also be found on lime (Tilia spp.) and poplar (Populus spp.). Widely used as a Christmas decoration, the use of mistletoe was recorded by Pliny the Elder who described the significance of mistletoe on oaks to Druids. The Tree Council along with the University of Oxford and Fera Science are looking for participants for a citizen science survey to determine how abundant mistletoe is, and where it grows.